In [1]:
!pip install pymorphy3
!pip install catboost
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 899.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.9 MB/s eta 0:00:00


In [2]:
import string
import re
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from pymorphy3 import MorphAnalyzer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, roc_curve
from sklearn.neighbors import NearestNeighbors
import torch
from torch import nn
from tqdm import tqdm
from catboost import CatBoostClassifier
import warnings

warnings.filterwarnings('ignore')

## Preprocess texts

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/My Drive/Term/texts_labeled.tsv'

In [5]:
df = pd.read_csv(path, sep='\t')

In [6]:
res = df.groupby('INPUT:comment')['OUTPUT:is_infringements'].apply(lambda row: row.value_counts().idxmax()).reset_index()
res

,INPUT:comment,OUTPUT:is_infringements
0,\r\nВРЕМЯ ПИТЬ ЧАЙ!\r\nBrusnikaTea «ЯГОДНЫЙ» -...,yes
1,\r\nВкусный чай зеленый листовой ягодами земля...,no
2,\r\nИван чай Глазова Гора – традиционный русск...,no
3,\r\nИщете идеальный подарок на Новый год? Пода...,no
4,\r\nНастоящий Молочный улун высший сорт из Кит...,yes
...,...,...
2003,Яркий фруктово-ягодный купаж на основе облепих...,yes
2004,Ярко выраженное восточное благоухание зелёного...,no
2005,пакетиках 14 матча день зеленый черный рождени...,no
2006,улун дыня - разновидность крупнолистового чая...,no


In [7]:
df[df['INPUT:comment'] == '\r\nВкусный чай зеленый листовой ягодами земляники. Земляничный улун подарочный. Улун китайский и ягоды сушеные (земляника сублимированная) возвратят Вас в лето и превратят чаепитие в праздник. Сублимированные ягоды и крупнолистовой зелёный чай укрепят здоровье. Содержит витамины D, C, E, K, марганец, фосфор, железо и йод. Чай заварной с натуральными добавками выводит шлаки и токсины из организма. Напиток можно использовать для детокса и похудения. Чай с ягодами имеет обволакивающий вкус и нежный аромат. Фруктовый чай натуральный антиоксидант, полезен для похудения, укрепляет костные ткани. Чайное ассорти изготовлено без искусственных красителей и не содержит ГМО. Чай подарочный ягодный микс в подарок женщине, мужчине. Чай с фруктами можно подать во время застолья на новогодний праздник. Оригинальный и нежный вкус наверняка понравится гостям. Состав: чай китайский улун, ягоды земляники, лист брусники, натуральный земляничный ароматизатор. После него Вы забудете про чай в пакетиках. сушеные ягоды, лепестки и травы - чайный напиток с натуральными витаминами и полезными минералами. Практичный, универсальный подарок с пользой для здоровья удивит и вызовет положительные эмоции, а также даст повод собраться с друзьями и родными за чашечкой чая. После него Вы забудете про чай в пакетиках. Тонизирующее, так и успокаивающее воздействие. ']

,INPUT:comment,OUTPUT:insult,OUTPUT:nonsense,OUTPUT:profanity,OUTPUT:advertising,OUTPUT:law_violation,OUTPUT:is_infringements,GOLDEN:insult,GOLDEN:nonsense,GOLDEN:profanity,...,GOLDEN:law_violation,GOLDEN:is_infringements,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started
4853,\r\nВкусный чай зеленый листовой ягодами земля...,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,https://platform.toloka.ai/task/42935533/00028...,00028f24ed--661febdf1c7df82555cadd30,00028f24ed--661ff1997d5f9b2330ba2f9c,5f271f4fb1d5d003e454617253cc57d8,APPROVED,2024-04-17T15:58:17.613
4901,\r\nВкусный чай зеленый листовой ягодами земля...,NaN,NaN,NaN,True,NaN,yes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,https://platform.toloka.ai/task/42935533/00028...,00028f24ed--661febdf1c7df82555cadd30,00028f24ed--661ff19bdaf59030df3c69c2,bfdb99f582393c82a7545545bc7044fb,APPROVED,2024-04-17T15:58:19.082
4953,\r\nВкусный чай зеленый листовой ягодами земля...,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,https://platform.toloka.ai/task/42935533/00028...,00028f24ed--661febdf1c7df82555cadd30,00028f24ed--661ff19cdaf59030df3c69f7,2a38e4d4e4a4736d579c9e52f33b9075,APPROVED,2024-04-17T15:58:20.522


In [8]:
names = {
    'INPUT:comment' : 'comment',
    'OUTPUT:is_infringements' : 'target'
}

In [9]:
res.rename(names, axis=1, inplace=True)

In [10]:
na = pd.DataFrame({'comment': ['Описания нет'], 'target': ['no']})


In [11]:
res = pd.concat([res, na], axis=0).reset_index().drop('index', axis=1)

In [197]:
# df['описание'] = df['описание'].fillna('Описания нет')
# df['бренд'] = df['бренд'].fillna('Бренда нет')

# df.info()

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
patterns = '[^0-9а-яА-ЯёЁ\s]+'
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]

            tokens.append(token)
    return tokens

In [14]:
def preprocess_text(row):

    text = str(row).lower()

    # deleting punctuation
    punc = str.maketrans('', '', string.punctuation)
    text_no_punct = text.translate(punc)

    # lemmatizing
    lematized_text = lemmatize(text_no_punct)

    return lematized_text

In [15]:
def target_labeler(s):
    if s == 'yes':
        return 1
    if s == 'avg':
        return 1
    else:
        return 0

In [16]:
preprocessed_df = res.copy()
preprocessed_df['comment'] = res['comment'].apply(preprocess_text)
preprocessed_df['target'] = preprocessed_df['target'].apply(lambda row: target_labeler(row))

## TFIDF + classical model

In [49]:
# train, test = train_test_split(preprocessed_df, random_state=34, train_size=0.8)

# print(train.shape)
# print(test.shape)

(1607, 2)
(402, 2)


In [50]:
# for sample in [train, test]:
#     print(sample[sample['target']==1].shape[0] / sample.shape[0])

0.3509645301804605
0.3407960199004975


In [51]:
# tfidf_1 = TfidfVectorizer(ngram_range=(1, 1))

In [52]:
# tfidf_train = tfidf_1.fit_transform(train['comment'])
# tfidf_test = tfidf_1.transform(test['comment'])

# print(tfidf_train.shape)
# print(tfidf_test.shape)

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
# lr = LogisticRegression(
#     solver='lbfgs',
#     random_state=33,
#     max_iter=10000,
#     n_jobs=-1
# )

In [ ]:
# lr.fit(tfidf_train, train['target'])

In [ ]:
# pred = lr.predict_proba(tfidf_test)

# roc_auc_score(test['target'], pred[:,0])

In [ ]:
# import matplotlib.pyplot as plt

# fpr, tpr, thres = roc_curve(test['target'], pred[:,0])

# plt.plot(fpr, tpr)

In [ ]:
# cb = CatBoostClassifier(iterations=500, learning_rate=0.01)

# cb.fit(tfidf_train, train['target'])

# pred = cb.predict_proba(tfidf_test)

# roc_auc_score(test['target'], pred[:,0])

In [ ]:
# print(accuracy_score(train['target'], cb.predict(tfidf_train)))
# print(accuracy_score(train['target'], lr.predict(tfidf_train)))

Опробовать сетки отсюда, если нет, то гг, придется что- то думать.
В противном случае генерация описаний или придумывание описательных фичей текста, которые могут помочь, вместо стандартных векторов

## Training embeddings

Придумать метрику оценивания док2века

In [17]:
# creating tagged data
tagged_data = [TaggedDocument(words=d, tags=[str(i)]) for i, d in enumerate(preprocessed_df['comment'])]

# creating model
model = Doc2Vec(vector_size=20, window=5, min_count=1, workers=4)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=10)

# receiving vecs
vectors = pd.DataFrame(model.infer_vector(doc.words) for doc in tagged_data)


In [18]:
def target_labeler(s):
    if s == 'yes':
        return 1
    if s == 'avg':
        return 1
    else:
        return 0

def counter(class_name):
    return preprocessed_df['target'].count()/preprocessed_df['target'][preprocessed_df['target']==class_name].count()

In [19]:
cb_df = pd.concat([vectors, preprocessed_df['target']], axis=1)
# cb_df['target'] = cb_df['target'].apply(lambda row: target_labeler(row))
# weights = [counter(0), counter(1), counter(2)]
cb_df


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,target
0,0.249353,0.407488,-0.028836,0.504527,-1.140328,-0.378027,-1.011132,0.701858,-0.476177,0.164415,...,1.030786,0.697098,-0.915580,0.339833,0.560148,1.054489,0.341003,-0.920360,-0.558324,1
1,0.264361,-0.254815,0.694006,0.097463,-0.540770,-1.005285,0.143063,0.246845,0.668529,0.404550,...,0.438121,0.454367,-0.284600,0.000602,0.391314,0.460228,0.255931,0.616159,-0.082114,0
2,0.396575,0.289177,0.038762,0.077473,-0.344482,0.071943,-0.351824,0.291307,-1.042082,0.358484,...,-0.042360,-0.385846,-0.070829,0.531166,-0.093685,0.223019,-0.086720,-0.185943,-0.566099,0
3,-0.180734,-0.243843,1.048579,0.143638,-0.244332,-1.223039,0.204592,-0.150266,0.100575,1.509365,...,0.782108,0.259131,0.413415,-0.214298,-0.682367,0.164967,0.555157,1.236762,-0.925236,0
4,0.025058,-0.202077,0.056562,0.762352,-0.425357,-0.731531,-0.630392,0.359895,-0.720309,-0.566724,...,-0.395844,0.352884,0.089227,1.039158,1.082849,0.584930,0.231830,-0.444211,-0.652356,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004,-0.059752,-0.171934,-0.020249,0.083559,-0.390745,-0.429569,-0.302085,0.097293,-0.432993,-0.170542,...,-0.150532,-0.055117,-0.100893,0.443921,0.305527,0.485646,-0.040702,-0.186336,-0.228632,0
2005,0.133246,-1.129759,0.898721,-0.556145,0.203925,0.349963,0.604167,-0.265094,0.936287,0.222510,...,-0.020598,0.322814,0.553916,-0.271050,-0.743434,-1.087996,1.073082,1.470464,-0.764718,0
2006,-0.178635,0.147474,-0.121518,0.482189,-0.712917,-0.397586,-0.557429,0.502940,-0.840400,-0.190748,...,-0.128986,0.299248,-0.639735,1.437275,0.688415,0.579135,0.293908,-0.260140,-0.481303,0
2007,0.209524,0.039326,0.116627,0.277146,-0.215641,-0.055390,-0.193576,0.160227,-0.197247,0.260790,...,0.071543,0.091299,-0.058789,0.104622,0.464914,0.165063,0.157320,-0.118159,-0.132950,0


In [20]:
cb_df.target.value_counts()

target
0    1308
1     701
Name: count, dtype: int64

In [21]:
X = cb_df.drop('target', axis=1)

positive_indices = [i for i, label in enumerate(cb_df['target']) if label == 1]
negative_indices = [i for i, label in enumerate(cb_df['target']) if label == 0]

positive_vectors = X.loc[positive_indices]
negative_vectors = X.loc[negative_indices]

KNN for filtering data

In [22]:
def knn_pos_filtering(positive_vectors, negative_vectors, thresh):

  knn_model_pos = NearestNeighbors(n_neighbors=3, algorithm='ball_tree')
  knn_model_pos.fit(positive_vectors)

  distances, indices = knn_model_pos.kneighbors(negative_vectors)

  result = set()
  for i, (s, dist, idx) in enumerate(zip(negative_indices, distances, indices)):
      if dist.mean() < thresh:
        result.add(s)
  return list(result)


In [23]:
pos_filtering_bad_idx = knn_pos_filtering(positive_vectors, negative_vectors, 1)
print(len(pos_filtering_bad_idx))
print(preprocessed_df.loc[pos_filtering_bad_idx]['target'].value_counts())

586
target
0    586
Name: count, dtype: int64


In [24]:
def knn_neg_filtering(positive_vectors, negative_vectors, thresh):

  knn_model_pos = NearestNeighbors(n_neighbors=3, algorithm='ball_tree')
  knn_model_pos.fit(negative_vectors)

  distances, indices = knn_model_pos.kneighbors(positive_vectors)

  result = set()
  for distance in range(len(distances)):
    for i in range(3):
      if distances[distance][i] < thresh:
        result.add(negative_indices[indices[distance][i]])
  return list(result)


In [25]:
neg_filtering_bad_idx = knn_neg_filtering(positive_vectors, negative_vectors, 1)
print(len(neg_filtering_bad_idx))
print(preprocessed_df.loc[neg_filtering_bad_idx]['target'].value_counts())

497
target
0    497
Name: count, dtype: int64


## Test bert for embeddings

In [26]:
device = 'cuda'

In [27]:
res['target'] = res['target'].apply(lambda x: target_labeler(x))
res['target'].value_counts()

target
0    1308
1     701
Name: count, dtype: int64

In [28]:
res.drop(neg_filtering_bad_idx)['target'].value_counts()
res.drop(pos_filtering_bad_idx)['target'].value_counts()


target
0    722
1    701
Name: count, dtype: int64

In [29]:
def preprocess_for_bert(df, batch=32):

  comments = res['comment'].to_list()
  labels = res['target'].to_list()
  tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

  # Токенизация данных
  input_ids = []
  attention_masks = []

  for comment in comments:
      encoded_data = tokenizer.encode_plus(
          comment,
          add_special_tokens=True,
          max_length=64,
          pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt'
      )
      input_ids.append(encoded_data['input_ids'])
      attention_masks.append(encoded_data['attention_mask'])

  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)

  train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=42, test_size=0.2)
  train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=42, test_size=0.2)

  train_data = TensorDataset(train_inputs, train_masks, train_labels)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch)

  validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
  validation_sampler = SequentialSampler(validation_data)
  validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch)

  return train_data, train_sampler, train_dataloader, validation_data, validation_sampler, validation_dataloader


BERT on pos filtering

In [30]:
train_data, train_sampler, train_dataloader, validation_data, validation_sampler, validation_dataloader = preprocess_for_bert(res.drop(pos_filtering_bad_idx), 32)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [31]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [32]:
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
model

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [33]:
train_loss_set = []
train_accuracy_set = []
validation_accuracy_set = []
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
model.dropout = nn.Dropout(0.2, inplace=False)
model.classifier = nn.Sequential(
    nn.Linear(768, 384),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(384,2),
)
model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)

model.train()
for epoch in range(10):
    total_train_accuracy = 0
    total_train_loss = 0

    for step, batch in enumerate(tqdm(train_dataloader, desc="Epoch {}".format(epoch))):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        model.zero_grad()

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        logits = outputs.logits
        loss = outputs.loss
        total_train_loss += loss.item()

        loss.backward()
        optimizer.step()


        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to(device).cpu().numpy()

        total_train_accuracy += flat_accuracy(logits, label_ids)

    avg_train_accuracy = total_train_accuracy / len(train_dataloader)
    avg_train_loss = total_train_loss / len(train_dataloader)
    train_accuracy_set.append(avg_train_accuracy)
    train_loss_set.append(avg_train_loss)

    print(" Average training accuracy: {0:.2f}".format(avg_train_accuracy))
    print(" Average training loss: {0:.2f}".format(avg_train_loss))


    model.eval()
    total_eval_accuracy = 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        logits = outputs.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.cpu().numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    validation_accuracy_set.append(avg_val_accuracy)
    print(" Validation Accuracy: {0:.2f}".format(avg_val_accuracy))



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 0:  51%|█████     | 26/51 [00:10<00:09,  2.59it/s]


KeyboardInterrupt: 

In [34]:
train_data, train_sampler, train_dataloader, validation_data, validation_sampler, validation_dataloader = preprocess_for_bert(res.drop(neg_filtering_bad_idx), 32)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [242]:
train_loss_set = []
train_accuracy_set = []
validation_accuracy_set = []
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
# model.dropout = nn.Dropout(0.2, inplace=False)
model.classifier = nn.Sequential(
    nn.Linear(768, 384),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(384,2),
)
model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)

model.train()
for epoch in range(10):
    total_train_accuracy = 0
    total_train_loss = 0

    for step, batch in enumerate(tqdm(train_dataloader, desc="Epoch {}".format(epoch))):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        model.zero_grad()

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        logits = outputs.logits
        loss = outputs.loss
        total_train_loss += loss.item()

        loss.backward()
        optimizer.step()


        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to(device).cpu().numpy()

        total_train_accuracy += flat_accuracy(logits, label_ids)

    avg_train_accuracy = total_train_accuracy / len(train_dataloader)
    avg_train_loss = total_train_loss / len(train_dataloader)
    train_accuracy_set.append(avg_train_accuracy)
    train_loss_set.append(avg_train_loss)

    print(" Average training accuracy: {0:.2f}".format(avg_train_accuracy))
    print(" Average training loss: {0:.2f}".format(avg_train_loss))


    model.eval()
    total_eval_accuracy = 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        logits = outputs.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.cpu().numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    validation_accuracy_set.append(avg_val_accuracy)
    print(" Validation Accuracy: {0:.2f}".format(avg_val_accuracy))



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 0: 100%|██████████| 51/51 [00:18<00:00,  2.74it/s]


 Average training accuracy: 0.66
 Average training loss: 0.65
 Validation Accuracy: 0.64


Epoch 1: 100%|██████████| 51/51 [00:18<00:00,  2.78it/s]


 Average training accuracy: 0.65
 Average training loss: 0.64
 Validation Accuracy: 0.64


Epoch 2: 100%|██████████| 51/51 [00:18<00:00,  2.79it/s]


...

 Average training accuracy: 0.95
 Average training loss: 0.17
 Validation Accuracy: 0.59


Epoch 8: 100%|██████████| 51/51 [00:18<00:00,  2.81it/s]


 Average training accuracy: 0.96
 Average training loss: 0.13
 Validation Accuracy: 0.58


Epoch 9: 100%|██████████| 51/51 [00:18<00:00,  2.80it/s]


 Average training accuracy: 0.95
 Average training loss: 0.12
 Validation Accuracy: 0.60


## Начинаем с простых методов

In [82]:
res

,comment,target
0,\r\nВРЕМЯ ПИТЬ ЧАЙ!\r\nBrusnikaTea «ЯГОДНЫЙ» -...,1
1,\r\nВкусный чай зеленый листовой ягодами земля...,0
2,\r\nИван чай Глазова Гора – традиционный русск...,0
3,\r\nИщете идеальный подарок на Новый год? Пода...,0
4,\r\nНастоящий Молочный улун высший сорт из Кит...,1
...,...,...
2004,Ярко выраженное восточное благоухание зелёного...,0
2005,пакетиках 14 матча день зеленый черный рождени...,0
2006,улун дыня - разновидность крупнолистового чая...,0
2007,"чай чёрный, рассыпной, масала, специи, саган д...",0


In [90]:
vocab_size = 30522
input_len = 300
hidden_size = 128
num_layers = 2
num_classes = 2
n_epochs = 5
learning_rate = 0.01

In [91]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_classes, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, num_classes)

    def forward(self, X):
        embedded_X = self.embedding(X)
        hidden_states = torch.zeros(self.num_layers, X.size(0), self.hidden_size).to(X.device)
        cell_states = torch.zeros(self.num_layers, X.size(0), self.hidden_size).to(X.device)
        out, _ = self.lstm(embedded_X, (hidden_states, cell_states))
        out = self.output_layer(out[:, -1, :])
        return out

In [92]:
model = LSTM(vocab_size, input_len, hidden_size, num_classes, num_layers)
model.to(device)

LSTM(
  (embedding): Embedding(30522, 300)
  (lstm): LSTM(300, 128, num_layers=2, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=2, bias=True)
)

In [93]:
X_train, X_test, y_train, y_test = train_test_split(res['comment'], res['target'], random_state=42, test_size=.2)

In [94]:
from collections import defaultdict

# Создание словаря слов
word_dict = defaultdict(lambda: len(word_dict))
PAD_IDX = word_dict['<PAD>']  # Паддинг для выравнивания длины последовательностей

# Создаем индекс слова для каждого уникального слова в корпусе
for text in X_train:  # предполагается, что `your_texts` это список списков слов
    for word in text:
        _ = word_dict[word]


In [95]:
# Функция для конвертации токенов в индексы с установленной длиной
def encode_text(text, max_length):
    encoded = [word_dict[word] for word in text]
    if len(encoded) < max_length:
        # Добавляем PAD_IDX для тех случаев, когда длина текста меньше max_length
        encoded += [PAD_IDX] * (max_length - len(encoded))
    return encoded[:max_length]  # Обрезаем текст до нужной длины, если он слишком длинный

max_length = 300  # Установленная длина последовательности
encoded_texts_train = [encode_text(text, max_length) for text in X_train]
encoded_texts_test = [encode_text(text, max_length) for text in X_test]



In [96]:
from tensorflow.keras import utils

In [97]:
# Преобразование данных в тензоры PyTorch
labels_train = torch.tensor(utils.to_categorical(y_train - 1, 2), dtype=torch.float32)
labels_test = torch.tensor(utils.to_categorical(y_test - 1, 2), dtype=torch.float32)

inputs_train = torch.tensor(encoded_texts_train, dtype=torch.long)
inputs_test = torch.tensor(encoded_texts_test, dtype=torch.long)

# Создание тензорного датасета
dataset_train = TensorDataset(inputs_train, labels_train)
dataset_test = TensorDataset(inputs_test, labels_test)

# Создание DataLoader
batch_size = 32  # Размер батча
train_data_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

In [99]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

LSTM(
  (embedding): Embedding(30522, 300)
  (lstm): LSTM(300, 128, num_layers=2, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=2, bias=True)
)

In [105]:
import torch

num_epochs = 5
for epoch in range(num_epochs):
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for inputs, targets in train_data_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()

        # Сохраняем потери для вывода среднего значения по эпохе
        total_loss += loss.item()

        # Расчёт точности
        predicted_probs = torch.sigmoid(outputs.squeeze())
        predicted_labels = (predicted_probs > 0.5).float()  # Пороговое значение 0.5 для бинарной классификации
        correct_predictions += (predicted_labels == targets).float().sum()
        total_predictions += targets.shape[0]

    accuracy = (correct_predictions / total_predictions).item()  # Расчет точности
    average_loss = total_loss / len(train_data_loader)
    print(f"Epoch {epoch + 1}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}")


ValueError: operands could not be broadcast together with shapes (32,) (64,) 